In [28]:
import re
import sys
import json

import urllib3
import configparser

import tensorflow_hub as hub
import tensorflow.compat.v1 as tf

from tqdm import tqdm
from pymongo import MongoClient
from bson.json_util import dumps
from elasticsearch import Elasticsearch, helpers

In [ ]:
print('Downloading pre-trained embeddings from tensorflow hub...')
tf.disable_eager_execution()
embed = hub.Module('../out/models/universal-sentence-encoder')
text_ph = tf.placeholder(tf.string)
embeddings = embed(text_ph)
print('Done.')

In [ ]:
print('Creating tensorflow session...')
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
session = tf.Session(config=config)
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())
print('Done.')

In [8]:
urllib3.disable_warnings()

config = configparser.ConfigParser()
config.read('../server.ini');

In [ ]:
# Elastic
es = Elasticsearch(
    config['ELASTIC']['path'],
    verify_certs=False
);

In [11]:
# Mongo
client = MongoClient(config['MONGODB']['path'])
swagger = client['swagger']['apis']

In [50]:
from bson import InvalidBSON


def load_json():
    cursor = swagger.find({})
    docs = swagger.count_documents({})
    
    for i in tqdm(range(docs), file=sys.stdout):
        try:
            document = json.loads(dumps(cursor.next()))
            nl_tags = r'(?:\'|\")(?:description|name|title|summary)(?:\'|\"):\s(?:\'|\")([^\'\"]+)(?:\'|\")'
            
            obj = {
                'id': str(document['_id']['$oid']),
                'name': str(document['_name']),
                'version': str(document['_version']) if not str(document['_version'].startswith('v')) else str(document['_version'][1:]),
                'embedding': session.run(embeddings, feed_dict={text_ph: [' '.join(re.findall(nl_tags, str(document)))]}).tolist()[0]
            }
            
            print(f'{i}/{docs}')
        except InvalidBSON:
            continue
        except StopIteration:
            continue
        
        # es.index(index='migration-test', body=obj)

In [ ]:
load_json()